In [4]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
# renderer for jupyter notebook
pio.renderers.default='notebook'
# renderer for vs code
# pio.renderers.default='vscode'
# renderer for colab
# pio.renderers.default='colab'
pio.templates.default = "plotly_dark"
import numpy as np
import statsmodels.api as sm
import datetime
from plotly.subplots import make_subplots
from ipywidgets import widgets,Layout

In [2]:
district=pd.read_csv(r'../data/processedfiles/rainfall-districts.csv')
koeppen=pd.read_csv(r'../data/processedfiles/rainfall-koe-regions.csv')
thornthwaite=pd.read_csv(r'../data/processedfiles/rainfall-tho-regions.csv')
climatic=pd.read_csv(r'../data/processedfiles/rainfall-climatic-regions.csv')

### Rainfall by region type

In [1]:
# dropdown widget
region_type = widgets.Dropdown(
    options=['District','Koeppen Climatic Regions',
             'Thornthwaite Climatic Regions', 'Climatic Region'],
    value='District', description='Region Type',layout=Layout(margin='20px 20px 20px 20px'))
container=widgets.HBox([region_type])

NameError: name 'widgets' is not defined

In [ ]:
traces=[go.Scatter(x=district['date'],y=district[i],name=i) for i in district.columns[3:]]
layout = go.Layout(title=dict(text='Rainfall by region type'),xaxis=dict(title='Years',rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall'))
go.Layout()
g = go.FigureWidget(data=traces, layout=layout)

In [5]:
def plot_fig(df, title):
    var = {'district': district,
           'koeppen': koeppen,
           'thornthwaite': thornthwaite,
           'climatic': climatic}
    df=var[df]
    with g.batch_update():
        col=list(df.columns.values)
        for i in range(len(g.data)):
            if i+3<len(col):
                g.data[i].y=df[col[i+3]]
                g.data[i].name=col[i+3]
                g.data[i].x=df['date']
            else:
                g.data[i].x=None
                g.data[i].y=None
                g.data[i].name=None

In [6]:
total_container=widgets.VBox([container,g])
display(total_container)

In [7]:
def response(change):
    val=region_type.value.split()[0].lower()
    plot_fig(val,region_type.value)

In [8]:
region_type.observe(response,names='value')

### Rainfall Complete Data


In [5]:
rain=pd.concat([district,koeppen.iloc[:,3:],climatic.iloc[:,3:],thornthwaite.iloc[:,3:]],axis=1)

In [6]:
rain=rain.set_index('date',drop=False)

In [8]:
rain['month']=rain.apply(lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%d").month,axis=1)
rain['day-of-month']=rain.apply(lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%d").day,axis=1)
rain=rain[list(rain.columns[:3])+list(rain.columns[48:50])+list(rain.columns[3:48])]

In [9]:
regions=list(rain.columns.values)[5:]

In [10]:
def plot_complete_data(df):
    fig = go.Figure()
    for i in df.columns[5:]:
        fig.add_trace(go.Scatter(
            x=df['date'],
            y=df[i],
            name=i))
    fig.update_layout(title_text='Rainfall based on all attributes',
                      xaxis_title='Time', yaxis_title='Rainfall', xaxis_rangeslider_visible=True)
    fig.show()

In [11]:
# plots all the attributes
# plot_complete_data(rain)

### Plot based on region type, and year range

In [29]:
traces_d=[]
traces_m=[]
for i in range(1957, 2013):
    temp = rain[rain['year'] == i]
    traces_d.append(go.Scatter(x=temp['day-number'],y=temp['ajmer'],name=str(i)))
    traces_m.append(go.Scatter(x=temp['month'],y=temp['ajmer'],name=str(i)))
layout_d = go.Layout(title=dict(text='Rainfall by region'),xaxis=dict(title='Days',rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall'))
layout_m=go.Layout(title=dict(text='Rainfall by region'),xaxis=dict(title='Months',rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall'))
g_d = go.FigureWidget(data=traces_d, layout=layout_d)
g_m=go.FigureWidget(data=traces_m, layout=layout_m)

In [33]:
def plot_sub_plots(region,year_start, year_end):
    df=rain[['date','year','day-number','month',region]]
    for i in range(1957, 2013):
        with g_d.batch_update():
            if year_start<=i<=year_end:
                temp = df[df['year'] == i]
                g_d.data[i-1957].y=temp[region]
                g_d.data[i-1957].name=str(i)
                g_d.data[i-1957].x=temp['day-number']
            else:
                g_d.data[i-1957].x=None
                g_d.data[i-1957].y=None
                g_d.data[i-1957].name=None
        with g_m.batch_update():
            if year_start<=i<=year_end:
                temp = df[df['year'] == i]
                g_m.data[i-1957].y=temp[region]
                g_m.data[i-1957].name=str(i)
                g_m.data[i-1957].x=temp['month']
            else:
                g_m.data[i-1957].x=None
                g_m.data[i-1957].y=None
                g_m.data[i-1957].name=None

In [31]:
def on_update_clicked(btn):
    plot_sub_plots(region.value,year_selector.value[0],year_selector.value[1])

In [34]:
region = widgets.Dropdown(
    options=regions,
    value=regions[0], description='Region',
layout=Layout(margin='20px 20px 20px 20px'))
year_selector = widgets.SelectionRangeSlider(
    options=list(rain['year'].unique()),
    index=(0, len(list(rain['year'].unique()))-1),
    description='Select Years', disabled=False,
layout=Layout(width='500px',margin='20px 20px 20px 20px'))
update_btn = widgets.Button(
    description='Update',
    disabled=False,
    button_style='success',
layout=Layout(margin='20px 20px 20px 20px'))
update_btn.on_click(on_update_clicked)
container = widgets.VBox([region, year_selector, update_btn,g_d,g_m])
display(container)

In [3]:
district.describe()


,year,day-number,ajmer,alwar,banswara,baran,barmer,bharatpur,bhilwara,bikaner,...,nagaur,pali,pratapgarh,rajsamand,sawai madhopur,sikar,sirohi,ganganagar,tonk,udaipur
count,20454.000000,20454.000000,19327.000000,19689.000000,20084.000000,8036.000000,14943.000000,19630.000000,20087.000000,18259.000000,...,20084.000000,20053.000000,1827.000000,7640.000000,14605.000000,20419.000000,20450.000000,20084.000000,18988.000000,20449.000000
mean,1984.501027,183.125257,1.231852,1.709242,2.361521,2.414630,0.729333,1.780907,1.629850,0.719182,...,1.085057,1.318173,2.631637,1.621219,1.880346,1.229352,2.155147,0.656415,1.622555,1.771191
std,16.163646,105.441220,5.695797,7.796654,11.518930,10.167191,4.500999,7.909387,6.124386,4.383149,...,4.663666,6.349870,10.741672,6.731750,8.814127,5.013550,10.101074,3.093403,6.379751,6.625139
min,1957.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1971.000000,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1984.500000,183.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1998.750000,274.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2012.000000,366.000000,294.000000,282.500000,363.000000,202.000000,185.000000,235.000000,182.580000,126.000000,...,125.120000,188.860000,173.400000,124.000000,175.000000,152.320000,313.100000,104.760000,182.800000,133.780000


### Avg rainfall per day of a year for every region

In [10]:
traces=[]
avg_day_year=rain.groupby(['day-number'])[regions].mean()
for i in avg_day_year.columns.values:
    traces.append(go.Scatter(x=avg_day_year.index,y=avg_day_year[i],name=str(i)))
layout= go.Layout(title=dict(text='Rainfall by region'),xaxis=dict(title='Days',rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall'))
g=go.FigureWidget(data=traces, layout=layout)

In [11]:
def on_update_clicked_avg(btn):
    avg_day_year=rain[rain['year'].isin([i for i in range(year_selector_avg.value[0],year_selector_avg.value[1]+1)])]
    avg_day_year=avg_day_year.groupby(['day-number'])[regions].mean()
    plot_avg(avg_day_year)

In [12]:
def plot_avg(df):
    with g.batch_update():
        for i in range(len(df.columns.values)):
            g.data[i].y=df[df.columns.values[i]]
            g.data[i].name=str(df.columns.values[i])
            g.data[i].x=df.index
        else:
            g.data[i].x=None
            g.data[i].y=None
            g.data[i].name=None

In [21]:
year_selector_avg = widgets.SelectionRangeSlider(
    options=list(rain['year'].unique()),
    index=(0, len(list(rain['year'].unique()))-1),
    description='Select Years', disabled=False,
    layout=Layout(width='500px', margin='20px 20px 20px 20px'))
update_btn = widgets.Button(
    description='Update',
    disabled=False,
    button_style='success',
    layout=Layout(margin='20px 20px 20px 20px'))
update_btn.on_click(on_update_clicked_avg)
container = widgets.VBox([year_selector_avg, update_btn, g])
display(container)